# Collaboration and Competition

---

Congratulations for completing the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control agents in a more challenging environment, where the goal is to train a team of agents to play soccer.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(1, 'scripts/')  # TODO: insert at 1, 0 is the script path (or '' in REPL)

from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
import shutil
import os
import pprint 
import matplotlib.pyplot as plt
%matplotlib inline

from d4pg_agent import D4PG_Agent
from params import Params
from logger import Logger

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Soccer.app"`
- **Windows** (x86): `"path/to/Soccer_Windows_x86/Soccer.exe"`
- **Windows** (x86_64): `"path/to/Soccer_Windows_x86_64/Soccer.exe"`
- **Linux** (x86): `"path/to/Soccer_Linux/Soccer.x86"`
- **Linux** (x86_64): `"path/to/Soccer_Linux/Soccer.x86_64"`
- **Linux** (x86, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86"`
- **Linux** (x86_64, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86_64"`

For instance, if you are using a Mac, then you downloaded `Soccer.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Soccer.app")
```

In [3]:
env = UnityEnvironment(file_name='Soccer_Windows_x86_64/Soccer.exe')
# env = UnityEnvironment(file_name='Soccer_Windows_x86_64/Soccer.exe', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we obtain separate brains for the striker and goalie agents.

In [4]:
# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]

['GoalieBrain', 'StrikerBrain']


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
num_g_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', num_g_agents)
num_s_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', num_s_agents)

# number of actions
g_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', g_action_size)
s_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', s_action_size)

# examine the state space 
g_states = env_info[g_brain_name].vector_observations
g_state_size = g_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(g_states.shape[0], g_state_size))
s_states = env_info[s_brain_name].vector_observations
s_state_size = s_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(s_states.shape[0], s_state_size))

Number of goalie agents: 2
Number of striker agents: 2
Number of goalie actions: 4
Number of striker actions: 6
There are 2 goalie agents. Each receives a state with length: 336
There are 2 striker agents. Each receives a state with length: 336


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [9]:
for i in range(2):                                         # play game for 2 episodes
    env_info = env.reset(train_mode=False)                 # reset the environment    
    g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
    s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
    g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
    s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
    while True:
        # select actions and send to environment
        g_actions = np.random.randint(g_action_size, size=num_g_agents)
        s_actions = np.random.randint(s_action_size, size=num_s_agents)
        actions = dict(zip([g_brain_name, s_brain_name], 
                           [g_actions, s_actions]))
        env_info = env.step(actions)                       
        
        # get next states
        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations
        
        # get reward and update scores
        g_rewards = env_info[g_brain_name].rewards  
        s_rewards = env_info[s_brain_name].rewards
        g_scores += g_rewards
        s_scores += s_rewards
        
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)  
        
        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states
        
        # exit loop if episode finished
        if done:                                           
            break
    print('Scores from episode {}: {} (goalies), {} (strikers)'.format(i+1, g_scores, s_scores))

Scores from episode 1: [1.00166669 1.00166669] (goalies), [-1.00166669 -1.00166669] (strikers)
Scores from episode 2: [ 0.29166667 -0.80833333] (goalies), [ 0.80833333 -0.29166667] (strikers)


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]

While training is taking place, statistics on agent performance are available from Tensorboard. To launch it use:
```python
cd <PROJECT_DIR>
tensorboard --logdir=runs
```

In [71]:
def d4pg(params=Params(), g_logger=Logger(), s_logger=Logger()):
    
    env_info = env.reset(train_mode=True)
    g_logger.initialize(g_agent, g_state_size, g_action_size)
    s_logger.initialize(s_agent, s_state_size, s_action_size)

#     try:
    
    start_eps = params.eps_to_resume_from if not params.restart_training else 1        
    for i_episode in range(start_eps, params.n_episodes+1):
        env_info = env.reset(train_mode=False)                 # reset the environment
        g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
        s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)            
        g_agent.reset()
        s_agent.reset()
        g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
        s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)

        #for t in range(params.max_t):    
        while True:

            # REPORT NAN STATES
            if np.isnan(g_states).any() or np.isnan(s_states).any():
                print('\nNaN found in states. Skipping this episode.')
                break

            # Perform actions from each agent's policy network (clipped actions [0, -1])
            g_actions = (np.argmax(g_agent.act(g_states[0])), np.random.randint(g_action_size))
            s_actions = (np.argmax(s_agent.act(s_states[0])), np.random.randint(s_action_size))
            actions = dict(zip([g_brain_name, s_brain_name], [g_actions, s_actions]))         
            env_info = env.step(actions)                   

            # get next states, rewards & dones
            g_next_states = env_info[g_brain_name].vector_observations         
            s_next_states = env_info[s_brain_name].vector_observations
            g_rewards = env_info[g_brain_name].rewards  
            s_rewards = env_info[s_brain_name].rewards
            done = np.any(env_info[g_brain_name].local_done) 

            # REPORT NAN ACTIONS
            if np.isnan(g_actions).any() or np.isnan(s_actions).any():
                print('\nNaN found in actions. Skipping this episode.')
                break

            if np.isnan(g_rewards).any() or np.isnan(s_rewards).any():
                print("NaN Reward Found! Skipping this episode.")
                g_rewards = np.where(np.isnan(g_rewards), -5.0, g_rewards)
                s_rewards = np.where(np.isnan(s_rewards), -5.0, s_rewards)
                break

            # Store in experience buffer & learn
            
            t = 1 # CHANGE ME!
            g_agent.step(g_states[0], g_actions[0], g_rewards[0], g_next_states[0], done, clear_nstep_buffer=(t==0))
            s_agent.step(s_states[0], s_actions[0], s_rewards[0], s_next_states[0], done, clear_nstep_buffer=(t==0))
            g_states = g_next_states
            s_states = s_next_states
            g_scores += g_rewards
            s_scores += s_rewards                

            # Break if game is over (time out / score)
            if done:   
                break 

        # Print crucial results for progress tracking
        if g_agent.memory_buffer_prefilled() and s_agent.memory_buffer_prefilled():

            g_logger.log_stats(i_episode, g_scores[0], g_agent.actor_loss, g_agent.critic_loss)
            s_logger.log_stats(i_episode, s_scores[0], s_agent.actor_loss, s_agent.critic_loss)
            print('\rEpisode {}: \Goalie Score: {:.2f} \Striker Score: {:.2f}'.format(i_episode, g_logger.scores_deque[-1], s_logger.scores_deque[-1]), end="")

            if i_episode % params.save_every == 0:
                s_logger.save_weights(i_episode)
                g_logger.save_weights(i_episode)

            if i_episode % params.print_every == 0:
                print("""\rEpisode {}: \t G_Actor_Loss: {:.2f} \tG_Critic_Loss: {:.2f} \tG_Average_Score: {:.2f} \n\t\t
                       S_Actor Loss: {:.2f} \tS_Critic Loss: {:.2f} \tS_Average Score: {:.2f}"""
                      .format(i_episode, np.mean(g_logger.actor_loss_deque), np.mean(g_logger.critic_loss_deque), np.mean(g_logger.scores_deque), 
                              i_episode, np.mean(s_logger.actor_loss_deque), np.mean(s_logger.critic_loss_deque), np.mean(s_logger.scores_deque)))
                #print('SCORES (this eps): ', score, '\n')
                #logger.plot_categorical_probs()

                if np.mean(g_logger.scores_deque) >= params.target_score and np.mean(s_logger.scores_deque) >= params.target_score:
                    print('\nEnvironment solved in {:d} episodes!\tS_Average Score: {:.2f} \tG_Average Score: {:.2f}'
                          .format(i_episode, np.mean(g_logger.scores_deque), np.mean(s_logger.scores_deque)))
                    if params.terminate_on_target_score:
                        break

    # Plot graphs & save final weights
    g_logger.save_weights(i_episode)
    s_logger.save_weights(i_episode)
    if params.plot_stats:
        g_logger.plot_stats()                       
        s_logger.plot_stats()
    g_logger.log_overall_perf_tb()
    s_logger.log_overall_perf_tb()

#     # Catch any exceptions (Esp with keyboard interrupts)
#     except BaseException as error:
#         print('\n\n==== An exception occurred: {}'.format(error))
#         g_logger.save_weights(i_episode)
#         s_logger.save_weights(i_episode)
#         if params.plot_stats:
#             g_logger.plot_stats()                       
#             s_logger.plot_stats()
#         g_logger.log_overall_perf_tb()
#         s_logger.log_overall_perf_tb()
    
    #return s_logger.scores_list

In [73]:
##################
# QUICK TEST
##################

# Params Override: Initialized in params.py
params = Params()
params.verbose = True
params.random_seed = 0
params.n_episodes=3
params.max_t = 50
params.print_every = 100
params.save_every = 100
params.log_weights_every = 1
params.prefill_memory_qty=0
params.terminate_on_target_score = True
params.plot_stats = True
params.lr_actor = 5e-4                  
params.lr_critic = 1e-3
params.checkpoint_actor_weights_dir = 'weights/checkpoint_actor/test'
params.checkpoint_critic_weights_dir = 'weights/checkpoint_critic/test'
params.vmin = -3
params.vmax = 3


#### MAIN #####
g_logger = Logger(params)
s_logger = Logger(params)
g_agent = D4PG_Agent(g_state_size, g_action_size, params=params)
s_agent = D4PG_Agent(s_state_size, s_action_size, params=params)
d4pg(params, g_logger, s_logger)


=============== NETWORKS ===============
actor_local Actor(
  (bn0): BatchNorm1d(336, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=336, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=512, out_features=4, bias=True)
)
actor_target Actor(
  (bn0): BatchNorm1d(336, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=336, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (bn2)

RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`